
# Assignment 1 – Data Cleaning & Preparation (Health Insurance Claims)

**Course:** Data Analytics for Actuarial Science  
**Week:** 3  
**Dataset:** `health_insurance_claims.csv`  
**Deliverables:**  
- Completed notebook (`.ipynb`)  
- Cleaned dataset (`health_insurance_claims_clean.csv`)  
- 1–2 page PDF summary of steps & justifications  

> **Objective:** Inspect, clean, and prepare an actuarial claims dataset for downstream modeling.


## 1. Setup & Load Data

In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

possible_paths = [
    'health_insurance_claims.csv',
    '/mnt/data/health_insurance_claims.csv'
]

data_path = None
for p in possible_paths:
    if os.path.exists(p):
        data_path = p
        break

if data_path is None:
    raise FileNotFoundError("Could not find 'health_insurance_claims.csv'. Please place it next to this notebook or adjust the path.")

df = pd.read_csv(data_path)
print(f'Loaded shape: {df.shape}')
df.head(10)


## 2. Initial Inspection (10 pts)
- Show dataset shape and first 10 rows  
- Identify numeric vs categorical variables  
- Summary statistics for numeric columns


In [ ]:

print('Shape:', df.shape)
display(df.head(10))
print('\nInfo:')
df.info()
print('\nDescribe (numeric):')
df.describe()
print('\nColumn dtypes:')
df.dtypes


## 3. Missing Values (20 pts)
- Count missing values per column  
- Impute:  
  - `age` → median  
  - `claim_amount` → (mean or median) *justify*  
  - `diagnosis_code` → mode  
- Show before/after missing summary


In [ ]:

missing_before = df.isna().sum()
print('Missing before:\n', missing_before)

# Impute age with median
if 'age' in df.columns:
    df['age'] = df['age'].fillna(df['age'].median())

# Impute claim_amount with your choice (mean or median) – default median here
if 'claim_amount' in df.columns:
    df['claim_amount'] = df['claim_amount'].fillna(df['claim_amount'].median())

# Impute diagnosis_code with mode (most frequent)
if 'diagnosis_code' in df.columns:
    mode_val = df['diagnosis_code'].mode(dropna=True)
    if len(mode_val) > 0:
        df['diagnosis_code'] = df['diagnosis_code'].fillna(mode_val.iloc[0])

missing_after = df.isna().sum()
print('\nMissing after:\n', missing_after)


## 4. Duplicates & Data Integrity (15 pts)
- Count duplicate `claim_id` (if present)  
- Drop duplicates and verify uniqueness


In [ ]:

if 'claim_id' in df.columns:
    dup_count = df.duplicated(subset='claim_id').sum()
    print('Duplicate claim_id rows:', dup_count)
    df = df.drop_duplicates(subset='claim_id')
    print('New shape after dropping dup claim_id:', df.shape)
    assert df['claim_id'].is_unique, "claim_id is not unique after drop_duplicates"
else:
    print("Column 'claim_id' not found; skipping this step.")


## 5. Outlier Detection & Treatment (25 pts)
- Use IQR on `claim_amount`  
- Decide to remove / cap (winsorize) / transform (e.g., log) *with justification*  
- Show before/after histograms


In [ ]:

# Plot BEFORE
if 'claim_amount' in df.columns:
    plt.figure()
    plt.hist(df['claim_amount'].dropna(), bins=30)
    plt.title('Claim Amount – Before')
    plt.xlabel('claim_amount'); plt.ylabel('count')
    plt.show()

    # IQR method
    q1 = df['claim_amount'].quantile(0.25)
    q3 = df['claim_amount'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    print(f'IQR bounds: [{lower:.2f}, {upper:.2f}]')

    # Example: capping (winsorization)
    df['claim_amount_capped'] = df['claim_amount'].clip(lower, upper)

    # Plot AFTER
    plt.figure()
    plt.hist(df['claim_amount_capped'].dropna(), bins=30)
    plt.title('Claim Amount – After Capping')
    plt.xlabel('claim_amount_capped'); plt.ylabel('count')
    plt.show()
else:
    print("Column 'claim_amount' not found; skipping outlier section.")


## 6. Transformations (20 pts)
- Encode categoricals: `gender`, `policy_type`, `payment_status`  
- Normalize/standardize `claim_amount` (or `claim_amount_capped`)  
- Explain why scaling may matter


In [ ]:

from sklearn.preprocessing import StandardScaler

categoricals = [c for c in ['gender', 'policy_type', 'payment_status', 'diagnosis_code'] if c in df.columns]
df_encoded = pd.get_dummies(df, columns=categoricals, drop_first=True)

target_col = 'claim_amount_capped' if 'claim_amount_capped' in df_encoded.columns else 'claim_amount'
if target_col in df_encoded.columns:
    scaler = StandardScaler()
    df_encoded[target_col + '_z'] = scaler.fit_transform(df_encoded[[target_col]])
else:
    print("No claim amount column available for scaling.")

print('Encoded columns added. New shape:', df_encoded.shape)
df_encoded.head()


## 7. Save Outputs & Brief Report (10 pts)
- Save cleaned dataset as `health_insurance_claims_clean.csv`  
- Write 1–2 pages summarizing: imputation choices, outlier handling, encoding, scaling, and any assumptions


In [ ]:

clean_path = 'health_insurance_claims_clean.csv'
df_encoded.to_csv(clean_path, index=False)
print(f'Saved cleaned dataset to: {clean_path}')


---

### Notes & Justifications (write here for the PDF report)
- **Missing values:**  
- **Duplicates:**  
- **Outliers:**  
- **Transformations:**  
- **Assumptions & limitations:**  

---
